In [1]:
from vis.visualization import visualize_cam, visualize_saliency
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import VGG16
from keras.applications.imagenet_utils import decode_predictions
from keras.models import load_model
import matplotlib.pyplot as plt
from pprint import pprint
import numpy as np
import cv2
import os
%matplotlib inline

Using TensorFlow backend.


In [2]:
# vars
# https://www.dropbox.com/sh/l3ezp9qyy5hid80/AAAjK6HdDScd_1rXASlsmELla?dl=0
# get 10 images from male and female
lfw_path = '/home/choppy/Desktop/CNN/visualization/dataset/lfw-dataset'
cropped_dataset = '/home/choppy/Desktop/CNN/visualization/dataset/cropped-lfw'

In [3]:
files = os.listdir(lfw_path)
sorted(files)
male, female = open(os.path.join(lfw_path, files[0]), 'r'), open(os.path.join(lfw_path, files[1]), 'r')
lfw_dataset = os.path.join(lfw_path, files[-1])
female = female.readlines()[:10]

wrong_m_list = [22, 219, 490, 680, 791, 826, 1183, 1201, 1287, 1615]
mmale = male.readlines()
male = []
for idx in wrong_m_list:
    male.append(mmale[idx])

print(lfw_dataset)
pprint(male)
pprint(female)

/home/choppy/Desktop/CNN/visualization/dataset/lfw-dataset/lfw
['Tommy_Shane_Steiner_0001.jpg\n',
 'Alberto_Acosta_0001.jpg\n',
 'Andy_Dick_0001.jpg\n',
 'Arminio_Fraga_0001.jpg\n',
 'Atal_Bihari_Vajpayee_0021.jpg\n',
 'Barry_Zito_0002.jpg\n',
 'Brandon_Boyd_0001.jpg\n',
 'Brendan_Hansen_0002.jpg\n',
 'Caio_Blat_0001.jpg\n',
 'Clint_Eastwood_0006.jpg\n']
['Claudia_Coslovich_0001.jpg\n',
 'Allison_Searing_0001\n',
 'Elizabeth_Hill_0001.jpg\n',
 'Erika_Reyes_0001.jpg\n',
 'Tatiana_Shchegoleva_0001.jpg\n',
 'AJ_Cook_0001.jpg\n',
 'Adelina_Avila_0001.jpg\n',
 'Adriana_Lima_0001.jpg\n',
 'Adriana_Perez_Navarro_0001.jpg\n',
 'Adrianna_Zuzic_0001.jpg\n']


In [4]:
# get list of images per class

male_images, female_images = [], []

for image_name_m, image_name_f in zip(male, female):
    attrs_m = image_name_m.split('_')[:-1]
    attrs_f = image_name_f.split('_')[:-1]
    folder_name_m = '_'.join(attrs_m)
    folder_name_f = '_'.join(attrs_f)
    
    image_path_m = os.path.join(lfw_dataset, folder_name_m, image_name_m)
    image_path_f = os.path.join(lfw_dataset, folder_name_f, image_name_f)
    
    if '.jpg' not in image_path_m:
        image_path_m = image_path_m.replace('\n', '')
        image_path_m += '.jpg'
    
    if '.jpg' not in image_path_f:
        image_path_f = image_path_f.replace('\n', '')
        image_path_f += '.jpg'
    
    male_images.append(image_path_m.replace('\n', ''))
    female_images.append(image_path_f.replace('\n', ''))

pprint(male_images)
print(len(male_images))
print()
pprint(female_images)
print(len(female_images))

['/home/choppy/Desktop/CNN/visualization/dataset/lfw-dataset/lfw/Tommy_Shane_Steiner/Tommy_Shane_Steiner_0001.jpg',
 '/home/choppy/Desktop/CNN/visualization/dataset/lfw-dataset/lfw/Alberto_Acosta/Alberto_Acosta_0001.jpg',
 '/home/choppy/Desktop/CNN/visualization/dataset/lfw-dataset/lfw/Andy_Dick/Andy_Dick_0001.jpg',
 '/home/choppy/Desktop/CNN/visualization/dataset/lfw-dataset/lfw/Arminio_Fraga/Arminio_Fraga_0001.jpg',
 '/home/choppy/Desktop/CNN/visualization/dataset/lfw-dataset/lfw/Atal_Bihari_Vajpayee/Atal_Bihari_Vajpayee_0021.jpg',
 '/home/choppy/Desktop/CNN/visualization/dataset/lfw-dataset/lfw/Barry_Zito/Barry_Zito_0002.jpg',
 '/home/choppy/Desktop/CNN/visualization/dataset/lfw-dataset/lfw/Brandon_Boyd/Brandon_Boyd_0001.jpg',
 '/home/choppy/Desktop/CNN/visualization/dataset/lfw-dataset/lfw/Brendan_Hansen/Brendan_Hansen_0002.jpg',
 '/home/choppy/Desktop/CNN/visualization/dataset/lfw-dataset/lfw/Caio_Blat/Caio_Blat_0001.jpg',
 '/home/choppy/Desktop/CNN/visualization/dataset/lfw-datas

In [5]:
# load and preprocess images

male_images = list(map(lambda x: cv2.imread(x), male_images))
male_images = list(map(lambda x: cv2.resize(x, (224, 224)), male_images))
male_images = list(map(lambda x: img_to_array(x), male_images))
male_images = list(map(lambda x: np.expand_dims(x, axis=0), male_images))

female_images = list(map(lambda x: cv2.imread(x), female_images))
female_images = list(map(lambda x: cv2.resize(x, (224, 224)), female_images))
female_images = list(map(lambda x: img_to_array(x), female_images))
female_images = list(map(lambda x: np.expand_dims(x, axis=0), female_images))


# read and process image
# image = cv2.imread(image_path)
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# image = cv2.resize(image, (224, 224))
# plt.imshow(image)
# image = img_to_array(image)
# image = np.expand_dims(image, axis=0)

In [6]:
# load model
# model = VGG16(include_top=True, weights='imagenet')
# model.summary()

model = load_model('../models/lr_1e-3_last8_layers.h5')

In [7]:
conv_layers = []
layers_by_id = {}

for idx, _layer in enumerate(model.layers):
    layers_by_id[idx] = _layer.name
    if "conv" in _layer.name:
        print(idx, "=>", _layer.name)
        conv_layers.append(idx)

print('conv_layers:', conv_layers, '\n')
pprint(layers_by_id)

0 => block1_conv1
1 => block1_conv2
3 => block2_conv1
4 => block2_conv2
6 => block3_conv1
7 => block3_conv2
8 => block3_conv3
10 => block4_conv1
11 => block4_conv2
12 => block4_conv3
14 => block5_conv1
15 => block5_conv2
16 => block5_conv3
conv_layers: [0, 1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16] 

{0: 'block1_conv1',
 1: 'block1_conv2',
 2: 'block1_pool',
 3: 'block2_conv1',
 4: 'block2_conv2',
 5: 'block2_pool',
 6: 'block3_conv1',
 7: 'block3_conv2',
 8: 'block3_conv3',
 9: 'block3_pool',
 10: 'block4_conv1',
 11: 'block4_conv2',
 12: 'block4_conv3',
 13: 'block4_pool',
 14: 'block5_conv1',
 15: 'block5_conv2',
 16: 'block5_conv3',
 17: 'block5_pool',
 18: 'flatten',
 19: 'fc1',
 20: 'fc2',
 21: 'predictions'}


In [8]:
# las primeras dos capas tiran error
# la primera es el input (obvio que tire error)
# la segunda es una capa convolutiva, por que tira error?

# solo para visualizar en jupyter
# for layer_id in conv_layers[2:]:
#     heat_map = visualize_cam(model, layer_id, None, image)
#     heat_map_saliency = visualize_saliency(model, layer_id, None, seed_input=image)

#     plt.figure(0)
#     plt.title("Grad-CAM {0:02}".format(layer_id) + " - " + layers_by_id[layer_id])
#     heat_map = cv2.resize(heat_map, (224, 224))
#     plt.imshow(heat_map)

#     # opencv2 channels are in BGR order. Transform the image to RGB to save it like a heatmap
#     # https://physiophile.wordpress.com/2017/01/12/why-opencv-uses-bgr-not-rgb/
#     destRGB = cv2.cvtColor(heat_map, cv2.COLOR_BGR2RGB)
#     destRGB = cv2.resize(destRGB, (224, 224))

#     plt.figure(1)
#     plt.title("Saliency")
#     plt.imshow(heat_map_saliency)

#     plt.show()

In [9]:
dim_output = (224*3, 224*3)

for layer_id in conv_layers[2:]:
    for image_m, image_f, image_name_m, image_name_f in zip(male_images, female_images, male, female):
        # female
        heat_map = visualize_cam(model, layer_id, None, image_f)
        heat_map_saliency = visualize_saliency(model, layer_id, None, seed_input=image_f)

        heat_map = cv2.resize(heat_map, dim_output)
        heat_map_saliency = cv2.resize(heat_map_saliency, dim_output)

        heat_map = cv2.cvtColor(heat_map, cv2.COLOR_BGR2RGB)
        heat_map_saliency = cv2.cvtColor(heat_map_saliency, cv2.COLOR_BGR2RGB)

        image_name_f = image_name_f.split('.')[0] # 0: name, 1: format
        cv2.imwrite("../heatmaps/grad-cam/female/"+layers_by_id[layer_id]+'_'+image_name_f+".png", heat_map)
        cv2.imwrite("../heatmaps/saliency/female/"+layers_by_id[layer_id]+'_'+image_name_f+".png", heat_map_saliency)
        
        # male
        print(image_name_m)
        heat_map = visualize_cam(model, layer_id, None, image_m)
        heat_map_saliency = visualize_saliency(model, layer_id, None, seed_input=image_m)

        heat_map = cv2.resize(heat_map, dim_output)
        heat_map_saliency = cv2.resize(heat_map_saliency, dim_output)

        heat_map = cv2.cvtColor(heat_map, cv2.COLOR_BGR2RGB)
        heat_map_saliency = cv2.cvtColor(heat_map_saliency, cv2.COLOR_BGR2RGB)

        image_name_m = image_name_m.split('.')[0] # 0: name, 1: format
        cv2.imwrite("../heatmaps/grad-cam/male/"+layers_by_id[layer_id]+'_'+image_name_m+".png", heat_map)
        cv2.imwrite("../heatmaps/saliency/male/"+layers_by_id[layer_id]+'_'+image_name_m+".png", heat_map_saliency)

Tommy_Shane_Steiner_0001.jpg

Alberto_Acosta_0001.jpg

Andy_Dick_0001.jpg

Arminio_Fraga_0001.jpg

Atal_Bihari_Vajpayee_0021.jpg

Barry_Zito_0002.jpg

Brandon_Boyd_0001.jpg

Brendan_Hansen_0002.jpg

Caio_Blat_0001.jpg

Clint_Eastwood_0006.jpg

Tommy_Shane_Steiner_0001.jpg

Alberto_Acosta_0001.jpg

Andy_Dick_0001.jpg

Arminio_Fraga_0001.jpg

Atal_Bihari_Vajpayee_0021.jpg

Barry_Zito_0002.jpg

Brandon_Boyd_0001.jpg

Brendan_Hansen_0002.jpg

Caio_Blat_0001.jpg

Clint_Eastwood_0006.jpg

Tommy_Shane_Steiner_0001.jpg

Alberto_Acosta_0001.jpg

Andy_Dick_0001.jpg

Arminio_Fraga_0001.jpg

Atal_Bihari_Vajpayee_0021.jpg

Barry_Zito_0002.jpg

Brandon_Boyd_0001.jpg

Brendan_Hansen_0002.jpg

Caio_Blat_0001.jpg

Clint_Eastwood_0006.jpg

Tommy_Shane_Steiner_0001.jpg

Alberto_Acosta_0001.jpg

Andy_Dick_0001.jpg

Arminio_Fraga_0001.jpg

Atal_Bihari_Vajpayee_0021.jpg

Barry_Zito_0002.jpg

Brandon_Boyd_0001.jpg

Brendan_Hansen_0002.jpg

Caio_Blat_0001.jpg

Clint_Eastwood_0006.jpg

Tommy_Shane_Steiner_0001

In [10]:
# load cropped dataset
files = os.listdir(cropped_dataset)
female = os.listdir(os.path.join(cropped_dataset, files[1]))
male = os.listdir(os.path.join(cropped_dataset, files[0]))

female = list(map(lambda x: os.path.join(cropped_dataset, 'female', x), female))
male = list(map(lambda x: os.path.join(cropped_dataset, 'male', x), male))

# load and preprocess images
male_images = list(map(lambda x: cv2.imread(x), male))
male_images = list(map(lambda x: cv2.resize(x, (224, 224)), male_images))
male_images = list(map(lambda x: img_to_array(x), male_images))
male_images = list(map(lambda x: np.expand_dims(x, axis=0), male_images))

female_images = list(map(lambda x: cv2.imread(x), female))
female_images = list(map(lambda x: cv2.resize(x, (224, 224)), female_images))
female_images = list(map(lambda x: img_to_array(x), female_images))
female_images = list(map(lambda x: np.expand_dims(x, axis=0), female_images))

print("[INFO] done!")

[INFO] done!


In [11]:
dim_output = (224*3, 224*3)

for layer_id in conv_layers[2:]:
    for image_m, image_f, image_name_m, image_name_f in zip(male_images, female_images, male, female):
        # female
        heat_map = visualize_cam(model, layer_id, None, image_f)
        heat_map_saliency = visualize_saliency(model, layer_id, None, seed_input=image_f)

        heat_map = cv2.resize(heat_map, dim_output)
        heat_map_saliency = cv2.resize(heat_map_saliency, dim_output)

        heat_map = cv2.cvtColor(heat_map, cv2.COLOR_BGR2RGB)
        heat_map_saliency = cv2.cvtColor(heat_map_saliency, cv2.COLOR_BGR2RGB)

        image_name_f = image_name_f.split('.')[0].split('/')[-1]
        cv2.imwrite("../heatmaps-cropped/grad-cam/female/"+layers_by_id[layer_id]+'_'+image_name_f+".png", heat_map)
        cv2.imwrite("../heatmaps-cropped/saliency/female/"+layers_by_id[layer_id]+'_'+image_name_f+".png", heat_map_saliency)
        
        # male
        heat_map = visualize_cam(model, layer_id, None, image_m)
        heat_map_saliency = visualize_saliency(model, layer_id, None, seed_input=image_m)

        heat_map = cv2.resize(heat_map, dim_output)
        heat_map_saliency = cv2.resize(heat_map_saliency, dim_output)

        heat_map = cv2.cvtColor(heat_map, cv2.COLOR_BGR2RGB)
        heat_map_saliency = cv2.cvtColor(heat_map_saliency, cv2.COLOR_BGR2RGB)

        image_name_m = image_name_m.split('.')[0].split('/')[-1]
        cv2.imwrite("../heatmaps-cropped/grad-cam/male/"+layers_by_id[layer_id]+'_'+image_name_m+".png", heat_map)
        cv2.imwrite("../heatmaps-cropped/saliency/male/"+layers_by_id[layer_id]+'_'+image_name_m+".png", heat_map_saliency)

print("[INFO] done!")

[INFO] done!


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________